# World Happiness

In [ ]:
import pandas as pd
url = 'https://happiness-report.s3.amazonaws.com/2022/Appendix_2_Data_for_Figure_2.1.xls'
data = pd.read_excel(url,x skipfooter=1)
data

In [ ]:
import plotly.express as px
px.bar(data, x='Country', y='Happiness score', title='World Happiness Report 2022', height=800)

In [ ]:

px.scatter(data, x='Country', y='Happiness score', size= "life expectancy", colour= social support  title='World Happiness Report 2022', height=800)

# Mapping the Data

We can run the code below to make a map of the countries compared to their happiness scores. 

In [ ]:
 
import folium

import geopandas as gpd

map= gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
map

map.plot(column='Happiness score', cmap='YlGnBu', edgecolor='black', legend=True)

# Questions to Explore

What factors do you think go into a a higher score of happiness? 

What factors do you think go into a a lower score of happiness? 

How do you think factors that influence quality of life affect the happiness score in a country? 